## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-22-20-52-55 +0000,nypost,Bessent teases ‘substantial’ Russia sanctions ...,https://nypost.com/2025/10/22/us-news/bessent-...
1,2025-10-22-20-48-07 +0000,nyt,Tesla Profit Falls 37% After It Cut Car Prices,https://www.nytimes.com/2025/10/22/business/te...
2,2025-10-22-20-47-41 +0000,nyt,Iceland Says It Is No Longer Mosquito Free,https://www.nytimes.com/2025/10/22/world/europ...
3,2025-10-22-20-47-00 +0000,wsj,Opinion | ‘Does India Even Have Any Cards?’,https://www.wsj.com/opinion/does-india-even-ha...
4,2025-10-22-20-45-00 +0000,wsj,"Las Vegas Sands Profit, Sales Jump as Macao, S...",https://www.wsj.com/business/earnings/las-vega...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2351/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
5,trump,44
98,israel,14
139,new,14
101,gaza,14
108,louvre,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
237,2025-10-22-10-00-00 +0000,nypost,Sen. John Kennedy talks Trump’s social media h...,https://nypost.com/2025/10/22/us-news/sen-john...,92
355,2025-10-21-21-57-30 +0000,bbc,Trump says he did not want 'wasted meeting' af...,https://www.bbc.com/news/articles/c4gjp73gp41o...,85
313,2025-10-22-02-42-00 +0000,wsj,Trump Officials Ratchet Up Pressure on Israel ...,https://www.wsj.com/world/middle-east/trump-of...,81
339,2025-10-21-23-57-52 +0000,bbc,Why Trump made breakthrough in Gaza but can't ...,https://www.bbc.com/news/articles/cx2y9je4d3go...,81
80,2025-10-22-18-46-50 +0000,wapo,U.S. kills two aboard alleged drug boat in Pac...,https://www.washingtonpost.com/national-securi...,78


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
237,92,2025-10-22-10-00-00 +0000,nypost,Sen. John Kennedy talks Trump’s social media h...,https://nypost.com/2025/10/22/us-news/sen-john...
35,64,2025-10-22-20-11-34 +0000,nypost,Top Wall Street investor warns Zohran Mamdani’...,https://nypost.com/2025/10/22/business/top-wal...
310,62,2025-10-22-03-11-49 +0000,nyt,Vance Lands in Israel as U.S. Tries to Shore U...,https://www.nytimes.com/2025/10/21/world/middl...
187,60,2025-10-22-13-31-00 +0000,wsj,Paul Ingrassia withdrew Tuesday from a Senate ...,https://www.wsj.com/politics/policy/paul-ingra...
32,50,2025-10-22-20-16-45 +0000,nypost,GM embraces AI with plans to use ‘eyes-off’ dr...,https://nypost.com/2025/10/22/business/gm-embr...
54,46,2025-10-22-19-35-20 +0000,nypost,America’s most infamous jewel thief slams Louv...,https://nypost.com/2025/10/22/lifestyle/infamo...
137,45,2025-10-22-16-24-00 +0000,wsj,"Fed officials, who are unable to receive U.S. ...",https://www.wsj.com/economy/central-banking/fe...
117,42,2025-10-22-17-04-00 +0000,wsj,Gucci-Owner Kering’s Sales Continue to Fall as...,https://www.wsj.com/business/retail/gucci-owne...
16,39,2025-10-22-20-36-29 +0000,nyt,Zelensky Says Russian Strike on School Shows P...,https://www.nytimes.com/2025/10/22/world/europ...
264,38,2025-10-22-09-00-00 +0000,wsj,The Republican National Committee heads into t...,https://www.wsj.com/politics/policy/democratic...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
